In [7]:
from typing import Tuple, List, Dict, Optional, Set
import heapq
import numpy as np

### Task 1

In [ ]:

maze: List[List[int]] = [
  [0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0]
]

def heuristic(source: Tuple[int, int], goal: Tuple[int, int]) -> int:
  return abs(source[0] - goal[0]) + abs(source[1] - goal[1])

goals: Set[Tuple[int, int]] = set([(4, 4), (2, 3), (0, 4)])

type DistanceDict = Dict[Tuple[int, int], Tuple[int, Optional[Tuple[int, int]]]]
def best_first_search(maze: List[List[int]], source: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
  dists: DistanceDict = {source: (0, None)}
  pq = [source]
  n, m = len(maze), len(maze[-1])

  def construct_path() -> List[Tuple[int, int]]:
    path = [goal]
    curr: Optional[Tuple[int, int]] = goal
    while curr:
      parent = dists[curr][1]
      path.append(parent)
      curr = parent
    return path[::-1][1:]

  while pq:
    r, c = heapq.heappop(pq)

    for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
      nr, nc = r + dr, c + dc
      if not (0 <= nr < n and 0 <= nc < m) or maze[nr][nc] == 1:
        continue

      f = heuristic((nr, nc), goal)
      if f < dists.get((nr, nc), (float('inf'), None))[0]:
        dists[(nr, nc)] = (f, (r, c))
        heapq.heappush(pq, (nr, nc))

  return construct_path()

def display_path(maze: List[List[int]], path: List[Tuple[int, int]]):
  for r, c in path:
    maze[r][c] = 2
  print(*map(lambda row: ' '.join(map(str, row)), maze), sep="\n", end="\n\n")

for goal in goals:
  path = best_first_search(maze, (0, 0), goal)
  display_path([row[:] for row in maze], path)

2 2 1 0 0
0 2 2 2 0
0 0 1 2 1
0 0 1 2 2
0 0 0 1 2

2 2 1 0 0
0 2 2 2 0
0 0 1 2 1
0 0 1 0 0
0 0 0 1 0

2 2 1 2 2
0 2 2 2 0
0 0 1 0 1
0 0 1 0 0
0 0 0 1 0



### Task 2

In [ ]:
class Node:
  def __init__(self, pos: Tuple[int, int], h_cost: int, g_cost: int):
    self.pos = pos
    self.h = h_cost
    self.g = g_cost
    self.f = h_cost + g_cost
    self.parent: Optional['Node'] = None
  
  def __lt__(self, obj: 'Node'):
    return self.f < obj.f
  
  def __str__(self):
    parent_str = self.parent.pos if self.parent else "None"
    return f"Node(pos={self.pos}, f={self.f}, h={self.h}, g={self.g}, parent={parent_str})"
  
  def __repr__(self):
    return self.__str__()

  def __hash__(self):
    return self.pos.__hash__()
  
N, M = 7, 7

def randomize_costs(maze: List[List[int]], k: int = 4, max_cost: int = 5):
  positions = list(zip(np.random.randint(0, N, k), np.random.randint(0, M, k)))
  for r, c in positions:
    cost = np.random.randint(1, max_cost+1)
    for dr, dc in [(0,0), (0,1), (1,0), (-1,0), (0,-1)]:
      nr, nc = r + dr, c + dc
      if 0 <= nr < N and 0 <= nc < M:
        maze[nr][nc] = cost
  return maze

def heuristic(source: Tuple[int, int], target: Tuple[int, int]):
  return abs(source[0] - target[0]) + abs(source[1] - target[1])

def dynamic_A_star(maze: List[List[int]], source: Tuple[int ,int], goals: Set[Tuple[int, int]]):
  pq: List[Node] = [Node(goal, 0, 0) for goal in goals]
  dists: Dict[Node, int] = {pq[0]: 0}
  print(pq)

  while pq:
    u = heapq.heappop(pq)
    r, c = u.pos

    for dr, dc in [(0,0), (0,1), (1,0), (-1,0), (0,-1)]:
      nr, nc = r + dr, c + dc
      if not (0 <= nr < N and 0 <= nc < M):
        continue
      h_cost = heuristic((nr, nc), source)
      g_cost = maze[nr][nc]
      if h_cost + g_cost < dists.get(u, float('inf')):
        dists[u] = int(h_cost + g_cost)
        v = Node((nr, nc), h_cost, g_cost)
        v.parent = u
        heapq.heappush(pq, v)
  print(dists)

np.random.seed(42)
maze = np.zeros((N, M), dtype=np.int32)
dynamic_A_star(randomize_costs(maze, 10), (0, 0), {(N-1, M-1), (N-1, 0), (0, M-1)})

[Node(pos=(6, 6), f=0, h=0, g=0, parent=None), Node(pos=(0, 6), f=0, h=0, g=0, parent=None), Node(pos=(6, 0), f=0, h=0, g=0, parent=None)]
{Node(pos=(6, 6), f=0, h=0, g=0, parent=None): 0}
